In [435]:
from feature.clip_make_dataset import make_data
from feature.after_dataset import accumulate
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [436]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [437]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)

In [438]:
# 증가율
result = []
result = pd.DataFrame()
for k in range(28):
    train1 = train[k*28:(k+1)*28]
    a = train1['predicted_weight_g'].pct_change()
    z = pd.DataFrame(a)
    result = pd.concat([result, z], axis=0)


In [439]:
result['DAT'] = train['DAT']

In [440]:
result = result.fillna(0)

In [441]:
weight = []
for i in range(28):
    weight.append(result[result['DAT']==i]['predicted_weight_g'].median())

In [442]:
weight_train = weight*28
weight_test = weight*5
train['weight'] = weight_train
test['weight'] = weight_test

In [443]:
weight_col = train.iloc[:, 2:16].columns

In [444]:
for idx, col in enumerate(weight_col):
    train['가중치'+col] = train[col]*train['weight']
    test['가중치'+col] = test[col]*test['weight']

In [445]:
train_rol = pd.read_csv('train_rol.csv')
test_rol = pd.read_csv('test_rol.csv')
train_rol = train_rol.drop(['DAT'], axis=1)
test_rol = test_rol.drop(['DAT'], axis=1)


In [446]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')

In [447]:
train_1 = pd.read_csv('./data/clip_preprocessing/clip_train_quantile0.25.csv')
test_1 = pd.read_csv('./data/clip_preprocessing/clip_test_quantile0.25.csv')
train_3 = pd.read_csv('./data/clip_preprocessing/clip_train_quantile0.75.csv')
test_3 = pd.read_csv('./data/clip_preprocessing/clip_test_quantile0.75.csv')
train_std = pd.read_csv('./data/clip_preprocessing/clip_train_std.csv')
test_std = pd.read_csv('./data/clip_preprocessing/clip_test_std.csv')
train_mid = pd.read_csv('./data/clip_preprocessing/clip_train_median.csv')
test_mid = pd.read_csv('./data/clip_preprocessing/clip_test_median.csv')
train_sem = pd.read_csv('./data/clip_preprocessing/clip_train_sem.csv')
test_sem = pd.read_csv('./data/clip_preprocessing/clip_test_sem.csv')

In [448]:
train_3.columns = [col+' q3' for col in train_3.columns]
test_3.columns = [col+' q3' for col in test_3.columns]
train_1.columns = [col+' q1' for col in train_1.columns]
test_1.columns = [col+' q1' for col in test_1.columns]
train_std.columns = [col+' std' for col in train_std.columns]
test_std.columns = [col+' std' for col in test_std.columns]
train_mid.columns = [col+' mid' for col in train_mid.columns]
test_mid.columns = [col+' mid' for col in test_mid.columns]
# train_sem.columns = [col+' sem' for col in train_sem.columns]
# test_sem.columns = [col+' sem' for col in test_sem.columns]

In [449]:
train = train.reset_index()
test  = test.reset_index()
train_3 = train_3.reset_index()
test_3 = test_3.reset_index()
train_1 = train_1.reset_index()
test_1 = test_1.reset_index()
train_std = train_std.reset_index()
test_std = test_std.reset_index()
train_mid = train_mid.reset_index()
test_mid = test_mid.reset_index()
# train_sem = train_sem.reset_index()
# test_sem = test_sem.reset_index()


In [450]:
train = pd.concat([train, train_1, train_3, train_std], axis=1)
test = pd.concat([test, test_1, test_3, test_std], axis=1)

train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)

# train = pd.concat([train, train_rol], axis=1)
# test = pd.concat([test, test_rol], axis=1)

train = pd.concat([train, p_train], axis=1)
test = pd.concat([test, p_test], axis=1)

In [451]:
# x_train = train.drop(['predicted_weight_g'], axis=1)
# y_train = train['predicted_weight_g']

# x_test = test.drop(['predicted_weight_g'], axis=1)
# y_test = test['predicted_weight_g']


In [452]:
# trian_col = x_train.columns
# test_col = x_test.columns
# from sklearn.preprocessing import MinMaxScaler

# scale = MinMaxScaler()

# x_train = scale.fit_transform(x_train)
# x_test = scale.transform(x_test)
# x_train = pd.DataFrame(data = x_train, columns=trian_col)
# x_test = pd.DataFrame(data = x_test, columns=test_col)
# train = pd.concat([x_train, y_train], axis=1)
# test = pd.concat([x_test, y_test], axis=1)


In [456]:
def model_train(train, test, model):
    '''
    train shape = (784, 18), test shape = (140, 18)
    model => sklearn api model
    '''

    X = train.drop(['predicted_weight_g', 'Case', 'obs_time', '6time', "weight"], axis=1)
    y = train['predicted_weight_g']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=113, shuffle=True)
    
    x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time', '6time', 'weight'], axis=1)
    
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_val_pred)**0.5
    print(f"validation rmse: {rmse}")
    
    x_train = train.drop(['predicted_weight_g', 'Case', 'obs_time', '6time', 'weight'], axis=1)
    y_train = train['predicted_weight_g']
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    x_test['predicted_weight_g'] = y_pred
    submit = x_test[['DAT', 'predicted_weight_g']]
    # submit['DAT'] = submit['DAT']
    all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
    for idx, test_path in enumerate(all_target_list):
        submit_df = pd.read_csv(test_path)
        submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
        submit_df.to_csv(test_path, index=False)

In [457]:
xgb = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50, task_type="GPU", random_seed=113)
lgbm = LGBMRegressor()
linear = LinearRegression()

model_train(train, test, cat)

Learning rate set to 0.037333
0:	learn: 40.7009187	total: 9.52ms	remaining: 9.51s
100:	learn: 7.5433219	total: 895ms	remaining: 7.96s
200:	learn: 5.4589579	total: 1.78s	remaining: 7.08s
300:	learn: 4.7564084	total: 2.67s	remaining: 6.2s
400:	learn: 4.3705575	total: 3.54s	remaining: 5.28s
500:	learn: 4.2365534	total: 4.39s	remaining: 4.37s
600:	learn: 4.0798789	total: 5.25s	remaining: 3.49s
700:	learn: 3.9463045	total: 6.12s	remaining: 2.61s
800:	learn: 3.8187688	total: 6.95s	remaining: 1.73s
900:	learn: 3.5696146	total: 7.81s	remaining: 859ms
999:	learn: 3.2902671	total: 8.66s	remaining: 0us
validation rmse: 8.773305651947913
Learning rate set to 0.039126
0:	learn: 40.3232198	total: 9.44ms	remaining: 9.43s
100:	learn: 6.9293470	total: 900ms	remaining: 8.01s
200:	learn: 5.1353031	total: 1.75s	remaining: 6.97s
300:	learn: 4.7045340	total: 2.6s	remaining: 6.05s
400:	learn: 4.3994760	total: 3.45s	remaining: 5.15s
500:	learn: 4.0307122	total: 4.31s	remaining: 4.29s
600:	learn: 3.8143218	tot

In [458]:
df_imp = pd.DataFrame({'imp':cat.feature_importances_}, index = cat.feature_names_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
"('시간당백색광량', 2.0)",0.000040
"('ec관측치', 4.0)",0.000191
"('ec관측치', 3.0)",0.000495
시간당백색광량 q3,0.000680
"('시간당적색광량', 2.0)",0.000834
...,...
내부온도관측치 q3,3.990960
월간누적적색광량,10.421555
월간누적백색광량,11.869067
DAT,19.149975
